Now that we have created the Embeddings and our Vectorstore is setup. We need to Get this knowlege in Realtime!

IN this Notebook:
- We will Take a query from User
- Convert it into Vector Embedding
- Search FAISS Database for matching Embeddings
- Get Embeddings and Index of Top K Matches.
- Out og these indexes, determine which are chunks and which ae superchunks
    - Create a Mongo Collection: chunk -> Superchunk
    - Get Superchunks of all fetched Embeddings
    - Now we have a list of all superchunks fetched and all their chunks.
    
    - if top5 had 3 sup chunks and 2 chunks,we can have all uptill 5 super chunks and all their chunks -> 50 Chunks\5 SuperChunks
- Rerank the Chunks
- Convert to  Text
- Pass to LLM along with Query

#### Imports:

In [1]:
## Load Environment Variables
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv(Path('C:/Users/erdrr/OneDrive/Desktop/Scholastic/NLP/LLM/RAG/FinsightRAG/.env'))

True

In [2]:
from src.embeddings.models import get_gpu_memory_stats
from src.embeddings.models import get_device
from src.embeddings.models import get_total_gpu_memory
from src.embeddings.models import get_embedding_model
from src.embeddings.models import check_model_is_available

In [3]:
from src.retrieval.retrieval import get_query_embeddings
from src.retrieval.retrieval import  search_embedding
from src.retrieval.retrieval import get_mega_chunks_by_indices
from src.retrieval.retrieval import get_all_chunks_for_mega_chunks_list

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.


# Retrieve

FAISS (Facebook AI Similarity Search) is a library developed by Facebook AI Research to facilitate efficient similarity search and clustering of dense vectors. It's particularly well-suited for tasks involving large-scale vector databases, like image or text embeddings. When it comes to fetching the top matching vectors for a given query embedding, FAISS provides various index types and search methods. Here's an overview of the general approach and specifically how to use HNSW (Hierarchical Navigable Small World) graphs, a method known for its efficiency in finding nearest neighbors in high-dimensional spaces.

https://github.com/facebookresearch/faiss

Faiss is built around the `Index` object. It encapsulates the set of database vectors, and optionally preprocesses them to make searching efficient. There are many types of indexes, we are going to use the simplest version that just performs brute-force L2 distance search on them: IndexFlatL2.

- Explain wht we need index
- Types of Index
- Flat Index vs HNSW



### General Approach for Fetching Top Matching Vectors

1. **Choose an Index Type:** FAISS offers several index types, each optimized for different kinds of data or search requirements. The choice of index impacts the efficiency and accuracy of the search.
   
2. **Index Building:** Once you've chosen an index type, you build the index by adding your database vectors to it. This step may involve training the index if it's a learned index (e.g., IVF indices).

3. **Searching:** After the index is built and populated, you can perform searches by passing query vectors to the index's search method. The method returns the IDs of the top matching vectors in the database along with their distances to the query vector.

### Using HNSW in FAISS

HNSW, or Hierarchical Navigable Small World graphs, is a graph-based approach that provides an excellent balance between search speed and accuracy. It's particularly effective for similarity searches in large-scale, high-dimensional datasets.

Here's a basic outline of how to use HNSW in FAISS:

1. **Index Creation:** First, create an HNSW index. FAISS provides the `IndexHNSWFlat` (and variants like `IndexHNSWScalarQuantizer`) for this purpose. The choice between them depends on your data and the trade-off between speed and memory usage you're willing to make.

    ```python
     import faiss
     d = 128  # Dimension of the vectors
     index = faiss.IndexHNSWFlat(d, M)  # M is a parameter defining the maximum number of connections for each element in the graph
    ```

2. **Index Population:** Add your vectors to the index. The vectors should be in a NumPy array.

    ```python
    xb = ...  # Your database vectors in a NumPy array
    index.add(xb)
    ```

3. **Setting HNSW Parameters:** Before searching, you might want to adjust HNSW parameters like `efConstruction` (which affects the index building time and quality) and `efSearch` (which influences the search time and accuracy).

    ```python
    index.hnsw.efConstruction = 40
    index.hnsw.efSearch = 16
    ```

4. **Searching:** Use the `search` method to find the top N matching vectors for your query vectors.

    ```python
    xq = ...  # Query vectors
    k = 10  # Number of nearest neighbors to retrieve
    D, I = index.search(xq, k)  # D are the distances, I are the indices of the nearest neighbors
    ```

### Additional Tips

- **Parameter Tuning:** HNSW parameters like `M`, `efConstruction`, and `efSearch` significantly impact both the quality of the search results and the performance. Experiment with these values based on your dataset and requirements.
- **Memory Management:** HNSW can be memory-intensive. If your dataset is very large, consider using a quantizer with HNSW or exploring other FAISS index types that offer better memory efficiency.
- **Batching Queries:** For efficiency, FAISS supports batching queries. If you have multiple query vectors, it's faster to search for them in a batch rather than one by one.

FAISS is a powerful tool, but getting the best performance out of it often requires experimentation with different index types, parameters, and optimizations specific to your use case.

TODO:  Study HNSW

- We are fetching topK based on Distance.
What are other ways we can fetch?

## KV Caching
Implementing Key-Value (KV) caching in Retrieval-Augmented Generation (RAG) systems involves creating a caching mechanism that stores the results of retrieval operations so that future queries can access pre-fetched information without needing to perform the retrieval step again. This can significantly improve the efficiency and speed of RAG systems, especially for queries that are frequently repeated or similar to previous queries. Below is a general approach to implementing KV caching in a RAG system:

### Step 1: Design the Cache Structure
First, decide on the structure of your cache. A simple and effective structure is a key-value store, where:
- **Key**: A representation of the query or input prompt. This could be the exact query text, a hashed version of it, or some form of embedding.
- **Value**: The retrieved documents or information along with any metadata necessary for the generation step.

### Step 2: Cache Integration Points
Identify the points in your RAG pipeline where the cache should be checked and updated:
- **Before Retrieval**: Check the cache using the current query as the key. If the query is found in the cache, use the cached results and skip the retrieval step.
- **After Retrieval**: Update the cache with the new query and its retrieved documents if the query was not found in the cache initially.

### Step 3: Implement Caching Logic
Implement the caching logic within your RAG workflow. This involves:
- **Checking the Cache**: Before performing retrieval, look up the query in the cache. If a match is found, use the cached data.
- **Updating the Cache**: After retrieval, if the data was not in the cache, add it with the current query as the key.

### Step 4: Cache Eviction and Management
Decide on a strategy for cache eviction and management to prevent the cache from growing indefinitely. Common strategies include:
- **Least Recently Used (LRU)**: Evict the least recently accessed items first.
- **Time-to-Live (TTL)**: Items in the cache expire after a certain time period.
- **Capacity**: Keep the cache size fixed, evicting items based on a predetermined strategy once the capacity is reached.

### Step 5: Serialization and Persistence (Optional)
For long-term efficiency, especially across system restarts, implement serialization and persistence for your cache. This means saving the cache to disk and loading it when the system starts.

### Example Pseudocode
```python
class RAGCache:
    def __init__(self, capacity=1000):
        self.cache = LRUCache(capacity)  # Assuming an LRUCache implementation is available

    def fetch_documents(self, query):
        # Check cache
        if query in self.cache:
            return self.cache[query]
        
        # Perform retrieval
        documents = perform_retrieval(query)
        
        # Update cache
        self.cache[query] = documents
        
        return documents

    # Additional methods for cache management, serialization, etc.
```

### Considerations
- **Cache Key Selection**: The choice of cache key is crucial. Simple queries might use the text directly, but for more complex inputs, consider embeddings or hashed values.
- **Cache Misses and Efficiency**: Monitor cache hit rates and adjust your caching strategy accordingly to ensure efficiency.
- **Security and Privacy**: Be mindful of what information is being cached, especially if sensitive data is involved.

Implementing KV caching in RAG systems can dramatically improve response times for repeated or similar queries, making your system more efficient and user-friendly.

In [5]:
query = "What is Credit Risk?"

In [6]:
embedding_vector = get_query_embeddings(query)

Loaded model and tokenizer from local directory: C:\Users\erdrr\OneDrive\Desktop\Scholastic\NLP\LLM\RAG\FinsightRAG\models\WhereIsAI\UAE-Large-V1\model
[INFO]: Generated Embedding of shape: (1, 1024)


In [7]:
indices,distance = search_embedding( embedding_vector, top_n=10,isL2Index=False)
print(f"[INFO]: Top {5} matching Indices: {indices}")

[INFO]: Top 5 matching Indices: [2971, 2887, 5219, 2888, 3355, 2973, 2948, 5368, 2889, 8613]


In [8]:
indices,distance = search_embedding( embedding_vector, top_n=10,isL2Index=True)
print(f"[INFO]: Top {5} matching Indices: {indices}")

[INFO]: Top 5 matching Indices: [35927, 35945, 35930, 56528, 35160, 2971, 35932, 39505, 56529, 2887]


- The top k indices can be either a mega chunk summary index or a chunk.
- If the index is a mega chunk, we will use it as it is, other wise we will get the mega chunk for the index recived.

- This will give us a list of all the mega chunks which are useful.

In [9]:
mega_chunks_list = get_mega_chunks_by_indices(indices)
print(f"[INFO]: Mega Chunks found in Top Indices: {mega_chunks_list}")

[INFO]: Mega Chunks found in Top Indices: [5219, 2971, 2887, 2889, 3355, 2972]


- Now we have a list of all the Mega Chunks
This could be even a single chunk, if all tok_n maching indices were from a single mega chunk and its chunks.
- Now for the list of Mega chunks, let's get all the chunk indexes which are part of this mega chunk.

In [10]:
retrieved_data,retrieved_chunks =get_all_chunks_for_mega_chunks_list(mega_chunks_list)
print(f"[INFO]: Total Chunks : {len(retrieved_chunks)}")

[INFO]: Total Chunks : 61


- We will get a maximum of `len(mega_chunks_list) * CHUNK_MULTIPLIER` chunks as mega chunks may have less chunk.

Now lets pass the chunks to a Reranker, and get the best results.

## Re-Rank

Now, out of all the retrieved embeddings, lets rerank them to get best embeddings.
we will pass all the 'chunks' (not mega chunks)
 - We need exact part of the text at the top, hence, the part of megachunk which contains the exact information will be placed at top.

- Types of Reranker
- Add some Papers
- Explain those in brief

In [ ]:
model_name = "mixedbread-ai/mxbai-rerank-large-v1"
model_save_path = Path(os.path.join(os.environ["MODELS_BASE_DIR"],model_name))

In [ ]:
check_model_is_available(model_name,model_save_path)

In [ ]:
def get_rerank_model(model_name):
    model_name = os.environ["RERANKER"]  
    model_save_path = Path(os.path.join(os.environ["MODELS_BASE_DIR"],model_name))

In [ ]:
def get_reranked_chunks(query, retrieved_chunks,return_documents=True, top_k=5):
    model_name = os.environ["RERANKER"]  
    model_save_path = Path(os.path.join(os.environ["MODELS_BASE_DIR"],model_name))
    model = CrossEncoder(model_save_path,device="cpu")
    reranked_chunks = model.rank(query, retrieved_chunks, return_documents=True, top_k=5)
    return reranked_chunks

In [25]:
from sentence_transformers import CrossEncoder
model = CrossEncoder(model_save_path,device="cpu")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [26]:
reranked_chunks = model.rank(query, retrieved_chunks, return_documents=True, top_k=5)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
reranked_chunks

# AUGMENT

- Now we have our data ready, we need to Augment this before we can pass it to an LLM for Generation.


- What is Augment?
- Is it Creating Prompt?

Creating a prompt to effectively leverage a local Large Language Model (LLM) for augmenting and reranking search results involves a few key steps. It should succinctly present the user's query, provide a structured format for the retrieved data, and pose the question clearly to ensure that the LLM understands what is expected in terms of augmentation and reranking. Here’s a structured approach to writing such a prompt:

1. **Introduction to the Task:** Start by briefly explaining the task to the LLM.
2. **User's Query:** Clearly state the original query from the user.
3. **Retrieved Data:** Present the top n results fetched from the FAISS using HNSW index. It's crucial to format this data clearly, perhaps in a list or a table, with each entry containing key details that the LLM might need to understand the context or content of the result. Include relevant details such as the title, a short snippet or summary, and any metadata like a relevance score or document ID.
4. **Question for LLM:** Specify what you need from the LLM. This could be augmenting the information with additional insights, explaining complex concepts in simpler terms, or reranking the results based on some criteria such as relevance to the query, comprehensiveness, clarity, etc.
5. **Instructions for Output:** Provide clear instructions on how you want the output structured. This can include asking for a ranked list, suggestions for further refinement, or additional questions to ask for deeper understanding.

### Sample Prompt:
**Task:** You are to augment the reranked results from an investment and finance terms database with additional insights and explanations, helping to provide a deeper understanding of each entry in relation to the user's query.

**User's Query:** "What are the implications of the Federal Reserve's interest rate changes on small businesses?"

**Data:**

Article 1: "Interest rate changes can significantly affect small businesses, especially in terms of loans and credit availability. This piece dives into the specifics."
Score: 93

Article 2: "The Federal Reserve adjusts interest rates to control inflation and stabilize the economy. This article explores the broad impacts of these adjustments."
Score: 89

Article 3: "Examining how various economic policies, including interest rate changes, influence the well-being of small businesses."
Score: 85

**Instructions for Augmentation:**

For each article, provide a detailed summary that extracts and highlights key points relevant to the user's query about the Federal Reserve's interest rate changes and their implications for small businesses.
Where possible, include any actionable insights or advice derived from each article that could benefit a small business owner.
Identify any gaps in the information provided by the articles that might require further research or clarification.
Question for LLM: Given the reranked articles and the user's query, can you augment the provided summaries with deeper insights, emphasizing how each article addresses the query? Additionally, suggest any complementary topics or questions that the user might find useful for a comprehensive understanding of the subject.

**Now based on above sample prompt, lets create "Data" part for our prompt.**

In [ ]:
articles_prompt = "\n\n"
for i,item in enumerate(reranked_chunks):
    articles_prompt += f"Article {i+1}: {item['text']} \nScore:{round(item['score']*100,4)} \n\n"
print(articles_prompt)

In [ ]:
base_prompt = f"""
Task: You are to augment the reranked results from an investment and finance terms database with additional insights and explanations, helping to provide a deeper understanding of each entry in relation to the user's query.

User's Query:{query}

{articles_prompt}
Instructions for Augmentation:

For each article, provide a detailed summary that extracts and highlights key points relevant to the user's query about the Federal Reserve's interest rate changes and their implications for small businesses.
Where possible, include any actionable insights or advice derived from each article that could benefit a small business owner.
Identify any gaps in the information provided by the articles that might require further research or clarification.
Question for LLM: Given the reranked articles and the user's query, can you augment the provided summaries with deeper insights, emphasizing how each article addresses the query? Additionally, suggest any complementary topics or questions that the user might find useful for a comprehensive understanding of the subject.

"""


In [ ]:
base_prompt = f"""Based on the following relevant articles items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible. 

\nRelevant Articles: {articles_prompt}
User query: {query}"""

In [ ]:
print(base_prompt)

In [21]:
def create_prompt(reranked_chunks):
    articles_prompt = "\n\n"
    for i,item in enumerate(reranked_chunks):
        articles_prompt += f"Article {i+1}: {item['text']} \nScore:{round(item['score']*100,4)} \n\n"
        
    base_prompt = f"""Based on the following relevant articles items, please answer the query.
    Give yourself room to think by extracting relevant passages from the context before answering the query.
    Don't return the thinking, only return the answer.
    Make sure your answers are as explanatory as possible. 

    \nRelevant Articles: {articles_prompt}
    User query: {query}"""
    dialogue_template = [
    {
        "role":"user",
        "content":base_prompt
    }]

    # Apply the chat template
    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                          tokenize=False,
                                          add_generation_prompt=True)
    return prompt

In [22]:
print(create_prompt(reranked_chunks))

NameError: name 'reranked_chunks' is not defined

## Generate

In [14]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers.utils import is_flash_attn_2_available

In [28]:
def get_llm():
    llm_model_id = os.environ["LLM_MODEL"]
    model_save_path = Path(os.path.join(os.environ["MODELS_BASE_DIR"],llm_model_id))
    use_quantization_config = True
    quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                        bnb_4but_compute_dtype=torch.float16)
    ## Define Attention
    if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0) >=8):
        attn_implementation = 'flash_attention_2'
    else:
        attn_implementation = "sdpa" # Scaled Dot Procust Attention
    print(f"[INFO]: Using Attention: {attn_implementation}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id,
                                 torch_dtype=torch.float16,
                                 quantization_config=quantization_config if use_quantization_config else None,
                                 low_cpu_mem_usage=False, # use as much memory as we can
                                 attn_implementation=attn_implementation)#.to(get_device())
    return llm_model,tokenizer

In [20]:
llm = get_llm(model_save_path,use_quantization_config=True)

[INFO]: Using Attention: sdpa


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def get_input_tokens(prompt,model):
# Tokenize the input Text (Prompt) - Turn it number nad send to GPU
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    input_ids = tokenizer(prompt,
                          return_tensors='pt')#.to("cpu")
    return input_ids

In [ ]:
def generate_llm_response(input_ids,llm_model,tokenizer, max_new_tokens=512):
    ## Generate Outputs from Local LLM
    outputs = llm_model.generate(**input_ids,
                            max_new_tokens = 256)
    outputs_decoded = tokenizer.decode(outputs[0])
    return outputs_decoded[outputs_decoded.find("<start_of_turn>model"):].strip("<start_of_turn>model")

In [ ]:
outputs_decoded = tokenizer.decode(outputs[0])
print(f"Model Output Decodes: \n{outputs_decoded}")

In [ ]:
outputs_decoded

In [ ]:
print(outputs_decoded[outputs_decoded.find("<start_of_turn>model"):].strip("<start_of_turn>model"))

# Lets RAG!!

In [ ]:
def create_prompt(reranked_chunks):
    """
    Augmentation: Augment the Retrived data and create prompt.
    """
    articles_prompt = "\n"
    for i,item in enumerate(reranked_chunks):
        articles_prompt += f"Article {i+1}: {item['text']} \nScore:{round(item['score']*100,4)} \n\n"
        
    base_prompt = f"""Answer the following question in a concise and informative manner:
    
{query}
    
Use the below Articles to answer the above question based on their relevance score. Create a combined answer from the articles:
{articles_prompt}
"""
    dialogue_template = [
    {
        "role":"user",
        "content":base_prompt
    }]

    # Apply the chat template
    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                          tokenize=False,
                                          add_generation_prompt=True)
    return prompt

In [24]:
query = "What is a record of revenue or expenses that have been earned or incurred but have not yet been recorded in the company's financial statements Called?"

In [27]:
embedding_vector = get_query_embeddings(query)
indices,distance = search_embedding(embedding_vector, top_n=10,isL2Index=True)
mega_chunks_list = get_mega_chunks_by_indices(indices)
retrieved_data,retrieved_chunks =get_all_chunks_for_mega_chunks_list(mega_chunks_list)
reranked_chunks = model.rank(query, retrieved_chunks, return_documents=True, top_k=5)

Loaded model and tokenizer from local directory: C:\Users\erdrr\OneDrive\Desktop\Scholastic\NLP\LLM\RAG\FinsightRAG\models\WhereIsAI\UAE-Large-V1\model
[INFO]: Generated Embedding of shape: (1, 1024)


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
augmented_query = create_prompt(reranked_chunks)
#print(augmented_query)

In [ ]:
%%time

## Generate Outputs from Local LLM
input_ids = tokenizer(augmented_query,
                      return_tensors='pt').to("cuda")
outputs = llm_model.generate(**input_ids,
                            max_new_tokens = 1024)
outputs_decoded = tokenizer.decode(outputs[0])
print(outputs_decoded[outputs_decoded.find("<start_of_turn>model"):].strip("<start_of_turn>model"))